# STEP 1 - creation of the java ontology

In [1]:
import sys

sys.path.append('/usr/local/lib/python3.8/site-packages')

from os import path as Path
import ast
from owlready2 import *
import javalang.tree
import rdflib
import rdflib.plugins.sparql as sq

# relative path of the subject code folder
chess_path              = './resources/android-chess/app/src/main/java/jwtc/chess/'
# relative path of the java tree structure
treePy_path             = './resources/tree.py'
# relative path of our populated ontology
populated_ontology_path = './computed/tree2.owl'
# relative path of our empty ontology
ontology_path           = './computed/tree.owl'
# relative path for queries
queries_path           = './queries/'

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
class Visitor(ast.NodeVisitor):

    def __init__(self, ontology):
        self.ontology = ontology

    def generic_visit(self, node):
        ast.NodeVisitor.generic_visit(self, node)
        with self.ontology as onto:
            if type(node) == ast.ClassDef:
                for obj in node.bases:
                    if obj.id == "Node":
                        types.new_class(node.name, (Thing,))
                    else:
                        types.new_class(node.name, (onto[obj.id],))

            elif type(node) == ast.Assign:
                for el in node.value.elts:
                    if el.s == "body" or el.s == "parameters":
                        types.new_class(el.s, (ObjectProperty,))
                    else:
                        types.new_class(
                            "jname" if el.s == "name" else el.s,
                            (DataProperty,))


In [3]:
def create_ontology(path, target_folder, name):
    with open(path, "r") as source:
        tree = ast.parse(source.read())

    ontology = get_ontology("http://Java_Ontology/" + name + ".owl")

    visitor = Visitor(ontology)
    visitor.visit(tree)
    if os.path.exists(target_folder):
        os.remove(target_folder)
    ontology.save(target_folder, format="rdfxml")
    
create_ontology(treePy_path, ontology_path, "JavaTree")

## STATS

`Axiom`							: 	4577
	
`Logical axiom count`			: 	3073
	
`Declaration axioms count`		: 	1504
	
`Class count`					: 	  78
	
`Object property count`			: 	   2
	
`Data property count`			: 	  65
	
`Individual count`				: 	1360

`SubClassOf`					: 	  83

`ClassAssertion`				: 	1360
	
`ObjectPropertyAssertion`		: 	1347
	
`DataPropertyAssertion`			: 	 283



# STEP 2-3 - populating the ontology

In [4]:
def append_fields(class_declaration, fields, ontology):
    for field in fields:
        o_fd = ontology['FieldDeclaration']()
        o_fd.jname = [field.name]
        class_declaration.body.append(o_fd)

def append_method(class_declaration, method, ontology):
    o_md = ontology['MethodDeclaration']()
    o_md.jname = [method.name]
    append_statements(o_md, method, ontology)
    append_parameters(o_md, method, ontology)
    class_declaration.body.append(o_md)
    
def append_constructor(class_declaration, method, ontology):
    o_con = ontology['ConstructorDeclaration']()
    o_con.jname = [method.name]
    append_statements(o_con, method, ontology)
    append_parameters(o_con, method, ontology)
    class_declaration.body.append(o_con)

def append_statements(md, method, ontology):
    for _, statement in method.filter(javalang.tree.Statement):
        if type(statement) != javalang.tree.Statement:
            s_type = statement.__class__.__name__
            s = ontology[s_type]()
            md.body.append(s)

def append_parameters(md, method, ontology):
    for _, statement in method.parameters:
        fp = ontology['FormalParameter']()
        md.parameters.append(fp)

In [5]:
def extract_class_declaration(source):
    class_declarations = []
    for file in os.listdir(source):
        if file.endswith('.java'):
            f_path = os.path.join(source, file)
            with open(f_path) as j_file:
                ast = javalang.parse.parse(j_file.read())
                for _, node in ast.filter(javalang.tree.ClassDeclaration):
                    class_declarations.append(node) 
    return class_declarations

def addClasses(ontology, class_declarations):
    for java_class in class_declarations:
        ontology_cd = ontology['ClassDeclaration']()
        ontology_cd.jname = [java_class.name]
        [append_fields(ontology_cd, f.declarators, ontology) for f in java_class.body if type(f) == javalang.tree.FieldDeclaration]
        [append_method(ontology_cd, m, ontology) for m in java_class.body if type(m) == javalang.tree.MethodDeclaration]
        [append_constructor(ontology_cd, c, ontology) for c in java_class.body if type(c) == javalang.tree.ConstructorDeclaration]
        
def add_instances(ontology_path, source_path):
    class_declarations = extract_class_declaration(source_path)
    ontology = get_ontology(ontology_path).load()
    with ontology:
        addClasses(ontology, class_declarations)
    if os.path.exists(populated_ontology_path):
        os.remove(populated_ontology_path)
    ontology.save(populated_ontology_path, format="rdfxml")
    


In [6]:
add_instances(ontology_path, chess_path)

## Stats


ClassDeclaration		:	11 

MethodDeclaration		:	152 

FieldDeclaration		:	105 

ConstructorDeclaratio	:	6 

FormalParameter			:	165 

AssertStatement			:	0 

BlockStatement			:	143 

BreakStatement			:	23 

CatchClause				:	8 

ContinueStatement		:	4 

DoStatement				:	2 

ForStatement			:	6 

IfStatement				:	125 

ReturnStatement			:	106 

StatementExpression		:	446 

SwitchStatement			:	8 

SynchronizedStatemen	:	1 

ThrowStatement			:	15 

TryStatement			:	8 

WhileStatement			:	10

# BAD SMELLS


In [7]:
def log(header, rows, logFunc):
    out = open(queries_path + "/" + header + ".txt", "w")
    for row in rows:
        out.write(logFunc(row))
    out.close()
    
if not os.path.exists(queries_path):
        os.makedirs(queries_path)
        
graph = default_world.as_rdflib_graph()

In [8]:
def longMethods(onto_name):
    return sq.prepareQuery(
 """SELECT ?className ?methodName ?statements (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                ?method tree:body ?statement .
                ?statement a/rdfs:subClassOf* tree:Statement .
            } GROUP BY ?method
            HAVING (COUNT(?statement) >= 20)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def longMethodsLog(row):
    return "CLASS: " + row.className + "\t METHOD: " + row.methodName + "\t STATEMENTS COUNT: " + row.tot + "\n"

In [9]:
def longConstructors(onto_name):
    return sq.prepareQuery(
        """SELECT ?className ?constructorName ?statements (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?constructor .
                ?constructor a tree:ConstructorDeclaration .
                ?constructor tree:jname ?constructorName .
                ?constructor tree:body ?statement .
                ?statement a/rdfs:subClassOf* tree:Statement .
            } GROUP BY ?constructor
            HAVING (COUNT(?statement) >= 20)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def longConstructorsLog(row):
    return "CLASS: " + row.className + "\t CONSTRUCTOR: " + row.methodName + "\t STATEMENTS COUNT: " + row.tot + "\n"

In [10]:
def largeClasses(onto_name):
    return sq.prepareQuery(
 """SELECT ?className ?methods (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
            } GROUP BY ?className
            HAVING (COUNT(?method) >= 10)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def largeClassesLog(row):
    return "CLASS: " + row.className + "\t METHODS COUNT: " + row.tot + "\n" 

In [11]:
def methodWithSwitch(onto_name):
    return sq.prepareQuery(
 """SELECT ?className ?methodName (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                ?method tree:body ?statement .
                ?statement a tree:SwitchStatement .
            } GROUP BY ?method
            HAVING (COUNT(?method) >= 1)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def methodWithSwitchLog(row):
    return "CLASS: " + row.className + "\t METHOD: " + row.methodName + "\t SWITCH COUNT: " + row.tot + "\n"

In [12]:
def constructorWithSwitch(onto_name):
    return sq.prepareQuery(
 """SELECT ?className ?constructorName (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?constructor .
                ?constructor a tree:ConstructorDeclaration .
                ?constructor tree:jname ?constructorName .
                ?constructor tree:body ?statement .
                ?statement a tree:SwitchStatement .
            } GROUP BY ?constructor
            HAVING (COUNT(?constructor) >= 1)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def constructorWithSwitchLog(row):
    return "CLASS: " + row.className + "\t CONSTRUCTOR: " + row.methodName + "\t SWITCH COUNT: " + row.tot + "\n"

In [13]:
def methodWithLongParameterList(onto_name):
    return sq.prepareQuery(
        """SELECT ?className ?methodName ?parameter (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                ?method tree:parameters ?parameter .
                ?parameter a tree:FormalParameter .
            } GROUP BY ?method
            HAVING (COUNT(?parameter) >= 5)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def MethodWithLongParameterListLog(row):
    return "CLASS: " + row.className + "\t METHOD: " + row.methodName + "\t PARAMETERS COUNT: " + row.tot + "\n"

In [14]:
def contructorWithLongParameterList(onto_name):
    return sq.prepareQuery(
 """SELECT ?className ?constructorName ?parameter (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?constructor .
                ?constructor a tree:ConstructorDeclaration .
                ?constructor tree:jname ?constructorName .
                ?constructor tree:parameters ?parameter .
                ?parameter a tree:FormalParameter .
            } GROUP BY ?constructor
            HAVING (COUNT(?parameter) >= 5)
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def contructorWithLongParameterListLog(row):
    return "CLASS: " + row.className + "\t CONSTRUCTOR: " + row.constructorName + "\t PARAMETERS COUNT: " + row.tot + "\n"

In [15]:
def dataClass(onto_name):
    return sq.prepareQuery(
 """SELECT ?className (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                FILTER (regex(?methodName, "set.*") || regex(?methodName, "get.*"))
            } GROUP BY ?className
        """,
 initNs = { "tree": "http://Java_Ontology/" + onto_name + ".owl#" })

def retriveAllMethods(onto_name):
    return sq.prepareQuery(
    """SELECT ?className (COUNT(*) AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
            } GROUP BY ?className
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

In [ ]:
res = graph.query(longMethods("JavaTree"))
log("LongMethods", res, longMethodsLog)

res = graph.query(longConstructors("JavaTree"))
log("LongConstructors", res, longConstructorsLog)
    
res = graph.query(largeClasses("JavaTree"))
log("LargeClasses", res, largeClassesLog)

res = graph.query(contructorWithLongParameterList("JavaTree"))
log("ConstructorWithLongParameterList", res, contructorWithLongParameterListLog)

res = graph.query(methodWithLongParameterList("JavaTree"))
log("MethodWithLongParameterList", res, MethodWithLongParameterListLog)

res = graph.query(constructorWithSwitch("JavaTree"))
log("ConstructorWithSwitch", res, constructorWithSwitchLog)

res = graph.query(methodWithSwitch("JavaTree"))
log("MethodsWithSwitch", res, methodWithSwitchLog)

resGS = graph.query(dataClass("JavaTree"))
resAll = graph.query(retriveAllMethods("JavaTree"))

out = open(queries_path + "/DataClasses.txt", "w")
for rowGS in resGS:
    for rowA in resAll:
        if rowGS.className == rowA.className:
            if rowGS.tot == rowA.tot:
                out.write("CLASS: " + rowGS.className + "\t GETTERS/SETTERS: " + rowGS.tot + "\n")
out.close()

## UNIT TEST 1

The purpose of tests in this section is to understand if in our ontology we correctly encoded some java instances, in particular ClassDeclarations, MethodDeclarations, FieldDeclarations, ConstructorDeclarations and VariableDeclarators

In order to test them, we just get the ontology we saved and we query it for each particular field, asserting that its name and its superclass(es) are encoded correctly


In [24]:
test_ontology_path           = './test/computed/tree_test.owl'
test_populated_ontology_path = './test/computed/tree_test2.owl'

create_ontology(treePy_path, test_ontology_path, "JavaTestTree")

In [25]:
onto = get_ontology(test_ontology_path).load()
cd = onto["ClassDeclaration"]

assert cd.name == "ClassDeclaration"
assert len(cd.is_a) == 1
assert cd.is_a[0].name == 'TypeDeclaration'

In [26]:
onto = get_ontology(test_ontology_path).load()
cd = onto["MethodDeclaration"]

assert cd.name == "MethodDeclaration"
assert len(cd.is_a) == 2
assert cd.is_a[0].name == 'Member'
assert cd.is_a[1].name == 'Declaration'

In [27]:
onto = get_ontology(test_ontology_path).load()
cd = onto["FieldDeclaration"]

assert cd.name == "FieldDeclaration"
assert len(cd.is_a) == 2
assert cd.is_a[0].name == 'Member'
assert cd.is_a[1].name == 'Declaration'

In [28]:
onto = get_ontology(test_ontology_path).load()
cd = onto["ConstructorDeclaration"]

assert cd.name == "ConstructorDeclaration"
assert len(cd.is_a) == 2
assert cd.is_a[0].name == 'Declaration'
assert cd.is_a[1].name == 'Documented'

In [29]:
onto = get_ontology(test_ontology_path).load()
cd = onto["VariableDeclarator"]

assert cd.name == "VariableDeclarator"
assert len(cd.is_a) == 1

In [30]:
onto = get_ontology(test_ontology_path).load()
cd = onto["VariableDeclarator"]

assert cd.name == "VariableDeclarator"
assert len(cd.is_a) == 1

In [31]:
onto = get_ontology(test_ontology_path).load()
cd = onto["Annotation"]

assert cd.name == "Annotation"
assert len(cd.is_a) == 1

In [32]:
onto = get_ontology(test_ontology_path).load()
cd = onto["ArrayInitializer"]

assert cd.name == "ArrayInitializer"
assert len(cd.is_a) == 1

In [33]:
onto = get_ontology(test_ontology_path).load()
cd = onto["CompilationUnit"]

assert cd.name == "CompilationUnit"
assert len(cd.is_a) == 1

In [34]:
onto = get_ontology(test_ontology_path).load()
cd = onto["ElementArrayValue"]

assert cd.name == "ElementArrayValue"
assert len(cd.is_a) == 1

In [35]:
onto = get_ontology(test_ontology_path).load()
cd = onto["ElementValuePair"]

assert cd.name == "ElementValuePair"
assert len(cd.is_a) == 1

In [36]:
onto = get_ontology(test_ontology_path).load()
cd = onto["EnhancedForControl"]

assert cd.name == "EnhancedForControl"
assert len(cd.is_a) == 1

In [37]:
onto = get_ontology(test_ontology_path).load()
cd = onto["EnumBody"]

assert cd.name == "EnumBody"
assert len(cd.is_a) == 1

In [38]:
onto = get_ontology(test_ontology_path).load()
cd = onto["ForControl"]

assert cd.name == "ForControl"
assert len(cd.is_a) == 1

In [39]:
onto = get_ontology(test_ontology_path).load()
cd = onto["Expression"]

assert cd.name == "Expression"
assert len(cd.is_a) == 1

In [40]:
onto = get_ontology(test_ontology_path).load()
cd = onto["Import"]

assert cd.name == "Import"
assert len(cd.is_a) == 1

In [41]:
onto = get_ontology(test_ontology_path).load()
cd = onto["InferredFormalParameter"]

assert cd.name == "InferredFormalParameter"
assert len(cd.is_a) == 1

In [42]:
onto = get_ontology(test_ontology_path).load()
cd = onto["Statement"]

assert cd.name == "Statement"
assert len(cd.is_a) == 1

In [43]:
onto = get_ontology(test_ontology_path).load()
cd = onto["SwitchStatementCase"]

assert cd.name == "SwitchStatementCase"
assert len(cd.is_a) == 1

In [44]:
onto = get_ontology(test_ontology_path).load()
cd = onto["Type"]

assert cd.name == "Type"
assert len(cd.is_a) == 1

In [45]:
onto = get_ontology(test_ontology_path).load()
cd = onto["TypeArgument"]

assert cd.name == "TypeArgument"
assert len(cd.is_a) == 1

In [46]:
onto = get_ontology(test_ontology_path).load()
cd = onto["TypeParameter"]

assert cd.name == "TypeParameter"
assert len(cd.is_a) == 1

In [47]:
onto = get_ontology(test_ontology_path).load()
cd = onto["VariableDeclarator"]

assert cd.name == "VariableDeclarator"
assert len(cd.is_a) == 1

In [48]:
onto = get_ontology(test_ontology_path).load()
cd = onto["EnumConstantDeclaration"]

assert cd.name == "EnumConstantDeclaration"
assert len(cd.is_a) == 2
assert cd.is_a[0].name == 'Declaration'

In [49]:
onto = get_ontology(test_ontology_path).load()
cd = onto["PackageDeclaration"]

assert cd.name == "PackageDeclaration"
assert len(cd.is_a) == 2
assert cd.is_a[0].name == 'Declaration'

## UNIT TESTS 2

The purpose of tests in this section is to understand if our functions to parse a java class and add its instances in our ontology are correct.

to do this we first load the java ontology we created, then we parse a custom class and we add it to the ontology using our functions, at that point we chack that every element of the class is stored correctly in the ontology


In [22]:
# here we parse the java class "class A { int x, y; }" and we check that after the routine we have two 
# field declarations, x and y belonging to class A

# get the ontology
onto = get_ontology(test_ontology_path).load()

# generating ast for test class
tree = javalang.parse.parse("class A { int x, y; }")

# storing each node declaration in an array
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   

# for each node in the array we add it to the ontology
with onto:
    addClasses(onto, cds)

# we save the populated ontology
onto.save(test_populated_ontology_path, format="rdfxml")

# we get the last class declaration, our test java class
a = onto['ClassDeclaration'].instances()[-1]

# we assert fields are stored correctly
assert a.body[0].is_a[0].name == 'FieldDeclaration'
assert a.body[0].jname[0] == 'x'
assert a.body[1].is_a[0].name == 'FieldDeclaration'
assert a.body[1].jname[0] == 'y'
assert a.jname[0] == 'A'

the process for this second test is exacly the same of the first one, but we just testi it with a richer class

<code>class Main { 
    int sum; 
    Main(){ 
        this(5, 2); 
    } 
    Main(int arg1, int arg2){ 
        this.sum = arg1 + arg2; 
    } 
    void display() { 
        System.out.println("Sum is: " + sum); 
    } 
    public static void main(String[] args){ 
        Main obj = new Main(); 
        obj.display(); 
    }
}</code>


we have multiple constructors and more fields and variables


In [23]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse('class Main { int sum; Main() { this(5, 2); } Main(int arg1, int arg2) { this.sum = arg1 + arg2; } void display() { System.out.println("Sum is: " + sum); } public static void main(String[] args) { Main obj = new Main(); obj.display(); } }')
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

a = onto['ClassDeclaration'].instances()[-1]
    
assert a.jname[0] == 'Main'
assert a.body[0].is_a[0].name == 'FieldDeclaration'
assert a.body[0].jname[0] == 'sum'
assert a.body[1].is_a[0].name == 'MethodDeclaration'
assert a.body[1].jname[0] == 'display'
assert a.body[2].is_a[0].name == 'MethodDeclaration'
assert a.body[2].jname[0] == 'main'
assert a.body[3].is_a[0].name == 'ConstructorDeclaration'
assert a.body[3].jname[0] == 'Main'
assert a.body[4].is_a[0].name == 'ConstructorDeclaration'
assert a.body[4].jname[0] == 'Main'

assert a.body[4].parameters[0].is_a[0].name == 'FormalParameter'
assert a.body[4].parameters[1].is_a[0].name == 'FormalParameter'

## UNIT TESTS 3

Even in this case we have the same process to test our functions, but here we specifically check the presence of return statements, something we did not checked in previous tests

In [24]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse('class A { int f() { return 0; } }')
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

a = onto['ClassDeclaration'].instances()[-1]
assert a.body[0].body[0].is_a[0].name == 'ReturnStatement'

## UNIT TESTS 4

Here we want to test the correctness of our queries, so what we do is similar in the structure of tests in section 2 and 3, but then at the end we just run our quesry in the populated ontology and check that we retrive the correct results, according to the specific classes we wrote

<code>
class A { 
    int f(int x) { 
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++; 
    } 
}
</code>

a class with a long method, more than 19 statements

In [25]:
# We load our ontology
onto = get_ontology(test_ontology_path).load()

# We parse a custom java function
tree = javalang.parse.parse("class A { int f(int x) { x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++; } }")

# storing each node declaration in an array
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   

# for each node in the array we add it to the ontology
with onto:
    addClasses(onto, cds)

# we save back our populated ontology
onto.save(test_populated_ontology_path, format="rdfxml")

# generate a graph
g = rdflib.Graph()
# load our ontology in the graph
g.load(test_populated_ontology_path)    
# run our specific query
res = g.query(longMethods("JavaTestTree"))
# check that we get the correct result
assert len(res) == 1

<code>
class MyClaas { 
    public MyClass(int x) { 
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++; 
    } 
}
</code>

a class with a long constructor, more than 19 statements

In [26]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public MyClass(int x) { x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++; } }")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(longMethods("JavaTestTree"))
assert len(res) == 1

<code>
class MyClaas { 
    public A(int x) { 
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++;
        x++; 
    } 
    public void AA(){} 
    public void B(){} 
    public void C(){} 
    public void D(){} 
    public void E(){} 
    public void F(){} 
    public void G(){} 
    public void H(){} 
    public void I(){} 
    public void J(){} 
    public void L(){} 
    public void K(){} 
    public void M(){}
}
</code>

A class with a lot of methods, more than 10

In [27]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public A(int x) { x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++; } public void AA(){} public void B(){} public void C(){} public void D(){} public void E(){} public void F(){} public void G(){} public void H(){} public void I(){} public void J(){} public void L(){} public void K(){} public void M(){} }")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(largeClasses("JavaTestTree"))
assert len(res) == 1

<code>
    class MyClaas { 
        public void A(int x){ 
            int day = 4; 
            switch (day) {
                case 1:
                    System.out.println(day); 
                    break; 
                case 2: 
                    System.out.println(day);  
                    break;
             } 
        }
    }
</code>

a class with a method containing at least one switch

In [28]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public void A(int x){ int day = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(methodWithSwitch("JavaTestTree"))
assert len(res) == 1

<code>
    class MyClaas { 
        public MyClaas(int x){ 
            int day = 4; 
            switch (day) {
                case 1:
                    System.out.println(day); 
                    break; 
                case 2: 
                    System.out.println(day);  
                    break;
             } 
        }
    }
</code>

a class with a contructor containing at least one switch

In [29]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public MyClaas(int x){ int day = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(constructorWithSwitch("JavaTestTree"))
assert len(res) == 1

<code>
class MyClaas { 
    public void A(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ 
        int day = 4; 
        int day1 = 4; 
        int d2ay = 4; 
        int day3 = 4;  
        int d22ay = 4; 
        int d2234ay = 4; 
        switch (day) {
            case 1: 
                System.out.println(day); 
                break; 
            case 2: 
                System.out.println(day);  
                break;
        } 
    }
}
</code>
    
A class with a method containg a lot of parameters, more than 4

In [30]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public void A(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ int day = 4; int day1 = 4; int d2ay = 4; int day3 = 4;  int d22ay = 4; int d2234ay = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(methodWithLongParameterList("JavaTestTree"))
assert len(res) == 1

<code>
class MyClaas { 
    public MyClaas(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ 
        int day = 4; 
        int day1 = 4; 
        int d2ay = 4; 
        int day3 = 4;  
        int d22ay = 4; 
        int d2234ay = 4; 
        switch (day) {
            case 1: 
                System.out.println(day); 
                break; 
            case 2: 
                System.out.println(day);  
                break;
        } 
    }
}
</code>
    
A class with a contructor containg a lot of parameters, more than 4

In [31]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public MyClaas(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ int day = 4; int day1 = 4; int d2ay = 4; int day3 = 4;  int d22ay = 4; int d2234ay = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(contructorWithLongParameterList("JavaTestTree"))
assert len(res) == 1

<code>
    class MyClaas { 
        public void setInt(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ 
            int day = 4; 
            int day1 = 4;
            int d2ay = 4; 
            int day3 = 4; 
            int d22ay = 4; 
            int d2234ay = 4; 
            switch (day) {
                case 1: 
                    System.out.println(day); 
                    break; 
                case 2: 
                    System.out.println(day);  
                    break;
                } 
            }
    }
</code>

A data class, classes with only getters and setters

In [32]:
onto = get_ontology(test_ontology_path).load()
tree = javalang.parse.parse("class MyClaas { public void setInt(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ int day = 4; int day1 = 4; int d2ay = 4; int day3 = 4;  int d22ay = 4; int d2234ay = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
cds = []
for _, node in tree.filter(javalang.tree.ClassDeclaration):
    cds.append(node)   
    
with onto:
    addClasses(onto, cds)

onto.save(test_populated_ontology_path, format="rdfxml")

g = rdflib.Graph()
g.load(test_populated_ontology_path)
res = g.query(dataClass("JavaTestTree"))
assert len(res) == 1